In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.models import load_model

In [2]:
df = pd.read_csv('Set.csv',delimiter=',')
data=df.drop_duplicates(keep='first')
data=df[df['sole_id'].astype(str).str.startswith('1')]
#print(data)

In [3]:
features = data.iloc[:, 2:-1].values 
#data.iloc[:, -1] = pd.to_numeric(data.iloc[:, -1], errors='coerce')
labels = data.iloc[:,-1].values - 1
scaler = StandardScaler()
features=scaler.fit_transform(features)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1,random_state=42)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_val = X_val.reshape(X_val.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_train.shape[1],1)


In [5]:
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

In [ ]:
verbose=0
model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=(12, 1)))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(12, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
loss, accuracy = model.evaluate(X_test, y_test)
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')
print(f'Train loss: {train_loss:.4f}')
print(f'Train accuracy: {train_accuracy:.4f}')


In [7]:
# import sys

# with open('output.txt','a') as f:
#     sys.stdout=f
#     print(f'Test loss: {loss:.4f}')
#     print(f'Test accuracy: {accuracy:.4f}')
#     print(f'Train loss: {train_loss:.4f}')
#     print(f'Train accuracy: {train_accuracy:.4f}')
    
#     sys.stdout=sys.__stdout__

In [8]:
model.save('pressmodel.h5')

In [ ]:
model=load_model('pressmodel.h5')
new_data= pd.read_csv('P_Labeled7.csv',delimiter=',')
new_features= new_data.iloc[:, 2:-1].values
new_features= scaler.transform(new_features)
new_features=new_features.reshape(new_features.shape[0], new_features.shape[1], 1)

predictions=model.predict(new_features)

predicted_labels=np.argmax(predictions, axis=1)
predicted_person=predicted_labels +1

print("predicted person:",predicted_person )

In [ ]:
print(len(predicted_person))
print(predicted_person[899:1000])